In [0]:
#https://arxiv.org/ftp/arxiv/papers/2003/2003.13145.pdf

In [2]:
!git clone https://github.com/mjkvaak/ImageDataAugmentor.git

Cloning into 'ImageDataAugmentor'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 263 (delta 54), reused 38 (delta 15), pack-reused 165
Receiving objects: 100% (263/263), 130.33 KiB | 373.00 KiB/s, done.
Resolving deltas: 100% (153/153), done.


In [3]:
from ImageDataAugmentor.image_data_augmentor import *

Using TensorFlow backend.


In [4]:
#Setup Kaggle DIR; copy json file ; chmod
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [5]:

#Install kaggle packages
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 81kB 1.4MB/s 
     |████████████████████████████████| 5.3MB 3.5MB/s 
     |████████████████████████████████| 112kB 25.1MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 112kB 32.1MB/s 


In [6]:
#!kaggle datasets list
!kaggle datasets list -s tawsifurrahman/covid19-radiography-database

ref                                          title                          size  lastUpdated          downloadCount  
-------------------------------------------  -----------------------------  ----  -------------------  -------------  
tawsifurrahman/covid19-radiography-database  COVID-19 Radiography Database   1GB  2020-03-28 23:36:00            124  


In [7]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

 99% 1.14G/1.15G [00:41<00:00, 33.8MB/s]
100% 1.15G/1.15G [00:41<00:00, 29.7MB/s]


In [0]:
%%capture
!unzip covid19-radiography-database.zip

In [0]:
%%capture
!pip install efficientnet

In [0]:
import glob
import pandas as pd
import os
import numpy as np

In [0]:

covid=os.listdir('/content/COVID-19 Radiography Database/COVID-19')
normal=os.listdir('/content/COVID-19 Radiography Database/NORMAL')
pneumonia=os.listdir('/content/COVID-19 Radiography Database/Viral Pneumonia')


In [12]:
cov=list(zip(covid,['covid']*(len(covid))))
covid_df = pd.DataFrame(cov,columns=['file','label']) 
covid_df.head()

,file,label
0,COVID-19 (48).png,covid
1,COVID-19(178).png,covid
2,COVID-19(180).png,covid
3,COVID-19 (19).png,covid
4,COVID-19(155).png,covid


In [13]:
norm=list(zip(normal,['normal']*(len(normal))))
normal_df = pd.DataFrame(norm,columns=['file','label']) 
normal_df.head()

,file,label
0,NORMAL (110).png,normal
1,NORMAL (360).png,normal
2,NORMAL (131).png,normal
3,NORMAL (662).png,normal
4,NORMAL (1009).png,normal


In [14]:
pneumonia=list(zip(pneumonia,['pneumonia']*(len(pneumonia))*2))
pneumonia_df = pd.DataFrame(pneumonia,columns=['file','label']) 
pneumonia_df.head()

,file,label
0,Viral Pneumonia (1067).png,pneumonia
1,Viral Pneumonia (1094).png,pneumonia
2,Viral Pneumonia (1287).png,pneumonia
3,Viral Pneumonia (1218).png,pneumonia
4,Viral Pneumonia (876).png,pneumonia


In [15]:
dataframe=pd.concat([covid_df,normal_df,pneumonia_df])
dataframe.head()

,file,label
0,COVID-19 (48).png,covid
1,COVID-19(178).png,covid
2,COVID-19(180).png,covid
3,COVID-19 (19).png,covid
4,COVID-19(155).png,covid


In [16]:
df = pd.get_dummies(dataframe['label'])
df.head()

,covid,normal,pneumonia
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [17]:
df = pd.concat([dataframe, df], axis=1)
df.head()

,file,label,covid,normal,pneumonia
0,COVID-19 (48).png,covid,1,0,0
1,COVID-19(178).png,covid,1,0,0
2,COVID-19(180).png,covid,1,0,0
3,COVID-19 (19).png,covid,1,0,0
4,COVID-19(155).png,covid,1,0,0


In [0]:
df=df.sample(frac=1)

In [0]:
!mkdir data

In [0]:
!cp -r /content/COVID-19\ Radiography\ Database/COVID-19/* data


In [0]:
!cp -r /content/COVID-19\ Radiography\ Database/NORMAL/* data


In [0]:
!cp -r /content/COVID-19\ Radiography\ Database/Viral\ Pneumonia/* data


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from albumentations import *


In [0]:
aug=Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
            
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=.1),
            Blur(blur_limit=3, p=.1),
        ], p=0.3),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.3),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomContrast(),
            RandomBrightness(),
        ], p=0.3),
        #HueSaturationValue(p=0.3),
    ], p=1)

In [0]:
#from keras_preprocessing.image import ImageDataGenerator
from ImageDataAugmentor.image_data_augmentor import *
data_gen= ImageDataAugmentor(
    #horizontal_flip=True,
    #vertical_flip=True,
    #rotation_range=90,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # zoom_range=.2,
    rescale=1/255,
    # fill_mode='nearest',
    # shear_range=0.2,
    # brightness_range=[0.2, 2.0],
    augment =aug)

In [0]:
img_shape=300
batch_size=16

In [0]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow as tf

In [38]:
from sklearn.model_selection import KFold
import gc

from sklearn.metrics import classification_report,accuracy_score,f1_score
classification_reports=[]
kf = KFold(n_splits=5)
history=[]
accuracy=[]
f1_scores=[]
fold=0
for train_index, test_index in kf.split(df):
  train=df.iloc[train_index,:]
  val=df.iloc[test_index,:]
  print('-----------fold {}--------------'.format(fold))
  train_generator=data_gen.flow_from_dataframe(train,directory='data',
                                                      target_size=(img_shape,img_shape),
                                                      x_col="file",
                                                      y_col=['covid','normal','pneumonia'],
                                                      class_mode='raw',
                                                      shuffle=True,
                                                      batch_size=batch_size)

  val_generator=data_gen.flow_from_dataframe(val,directory='data',
                                                        target_size=(img_shape,img_shape),
                                                        x_col="file",
                                                        y_col=['covid','normal','pneumonia'],
                                                        class_mode='raw',
                                                        shuffle=False,
                                                        batch_size=batch_size)
  #train_generator.next()[0].shape,train_generator.next()[1].shape
  import efficientnet.tfkeras as efn
  model =efn.EfficientNetB4(weights ='noisy-student', include_top=False, input_shape = (img_shape,img_shape,3))
    
  x = model.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.3)(x)
  x = Dense(128, activation="relu")(x)
  #x = Dropout(0.3)(x)
  x = Dense(64, activation="relu")(x)
  predictions = Dense(3, activation="softmax")(x)
  model = Model(inputs=model.input, outputs=predictions)

  def custom_loss(y_true, y_pred):
      return tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)
  model.compile(optimizer=Adam(0.0001), loss=custom_loss, metrics=['accuracy'])

  results = model.fit(train_generator,epochs=25,
                              steps_per_epoch=train_generator.n/batch_size,
                              validation_data=val_generator,
                             validation_steps=val_generator.n/batch_size,
                              callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=3, min_lr=0.000001),
                                         ModelCheckpoint('model_{}.hdf5'.format(fold), save_best_only=True, monitor='val_loss', mode='min')
                                         ])
  model.load_weights('model_{}.hdf5'.format(fold))
  val_generator.reset()
  y_pred=model.predict(val_generator,steps=val_generator.n/batch_size,verbose=1)
  y_pred=y_pred.round().astype(int)
  y_true=val.iloc[:,2::]
  classification_reports.append(classification_report(y_true, y_pred,target_names=['covid','normal','pneumonia']))#
  accuracy.append(accuracy_score(y_true,y_pred))
  f1=f1_score(y_true,y_pred,average='macro')
  print('f1 score is ', f1)
  f1_scores.append(f1)
  fold+=1
  del model
  tf.keras.backend.clear_session()
  gc.collect()
  

-----------fold 0--------------
Found 2324 validated image filenames.
Found 581 validated image filenames.
Epoch 1/25
146/145 [==============================] - 126s 864ms/step - loss: 0.7053 - accuracy: 0.7638 - val_loss: 0.6453 - val_accuracy: 0.8399 - lr: 1.0000e-04
Epoch 2/25
146/145 [==============================] - 128s 874ms/step - loss: 0.4743 - accuracy: 0.9152 - val_loss: 0.5462 - val_accuracy: 0.8623 - lr: 1.0000e-04
Epoch 3/25
146/145 [==============================] - 127s 867ms/step - loss: 0.4392 - accuracy: 0.9243 - val_loss: 0.4984 - val_accuracy: 0.8881 - lr: 1.0000e-04
Epoch 4/25
146/145 [==============================] - 123s 842ms/step - loss: 0.4218 - accuracy: 0.9290 - val_loss: 0.4285 - val_accuracy: 0.9312 - lr: 1.0000e-04
Epoch 5/25
146/145 [==============================] - 122s 836ms/step - loss: 0.4096 - accuracy: 0.9432 - val_loss: 0.3795 - val_accuracy: 0.9535 - lr: 1.0000e-04
Epoch 6/25
146/145 [==============================] - 123s 839ms/step - loss: 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


f1 score is  0.9861540631747995
-----------fold 2--------------
Found 2324 validated image filenames.
Found 581 validated image filenames.
Epoch 1/25
146/145 [==============================] - 130s 891ms/step - loss: 0.6880 - accuracy: 0.7874 - val_loss: 0.8277 - val_accuracy: 0.6282 - lr: 1.0000e-04
Epoch 2/25
146/145 [==============================] - 121s 830ms/step - loss: 0.4942 - accuracy: 0.9053 - val_loss: 0.5987 - val_accuracy: 0.8244 - lr: 1.0000e-04
Epoch 3/25
146/145 [==============================] - 130s 891ms/step - loss: 0.4449 - accuracy: 0.9277 - val_loss: 0.5282 - val_accuracy: 0.8744 - lr: 1.0000e-04
Epoch 4/25
146/145 [==============================] - 126s 860ms/step - loss: 0.4185 - accuracy: 0.9380 - val_loss: 0.4024 - val_accuracy: 0.9432 - lr: 1.0000e-04
Epoch 5/25
146/145 [==============================] - 126s 861ms/step - loss: 0.4225 - accuracy: 0.9337 - val_loss: 0.3676 - val_accuracy: 0.9604 - lr: 1.0000e-04
Epoch 6/25
146/145 [==========================

In [39]:
accuracy

[0.9827882960413081,
 0.9845094664371773,
 0.9724612736660929,
 0.9724612736660929,
 0.9741824440619621]

In [40]:
print(np.mean(accuracy))

0.9772805507745266


In [41]:
f1_scores

[0.9815409599130224,
 0.9861540631747995,
 0.9772288842920812,
 0.9806688882043814,
 0.978913715784171]

In [42]:
np.mean(f1_scores)

0.980901302273691

In [43]:

for i in range(5):
  print('--------------FOLD {}------------------'.format(i))
  print(classification_reports[i])

--------------FOLD 0------------------
              precision    recall  f1-score   support

       covid       1.00      0.96      0.98        47
      normal       0.97      0.99      0.98       265
   pneumonia       0.99      0.98      0.99       269

   micro avg       0.98      0.98      0.98       581
   macro avg       0.99      0.98      0.98       581
weighted avg       0.98      0.98      0.98       581
 samples avg       0.98      0.98      0.98       581

--------------FOLD 1------------------
              precision    recall  f1-score   support

       covid       1.00      0.98      0.99        43
      normal       0.98      0.99      0.98       259
   pneumonia       0.99      0.98      0.99       279

   micro avg       0.99      0.98      0.99       581
   macro avg       0.99      0.98      0.99       581
weighted avg       0.99      0.98      0.99       581
 samples avg       0.98      0.98      0.98       581

--------------FOLD 2------------------
             